In [4]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Device for Train : {device}")

Device for Train : cpu


In [7]:
import os

In [8]:
os.listdir('./')

['.ipynb_checkpoints',
 '2022-10-27_call_model.pt',
 'checkpoint-2500',
 'checkpoint-3000',
 'logs',
 '연락처학습데이터(1025).csv',
 '자기소개글_연락처_모델_학습(1025).ipynb']

In [6]:
data = pd.read_csv('./연락처학습데이터(1025).csv')
data

,conts,label
0,XWUIUNWICCQOEEXQXZQZQZIJVTMIECOEPWXQ,0
1,fammer ismylifeyazxcvya0 00 000 08000 000000ya...,0
2,11131 카통 9999999999999999999999999999999999999...,0
3,Jdididi아야야야야어어29929292냐냐냐냐냐냐냐냐냐냐나나나난929터터ㅤㅌㅓㅌ,1
4,01067120499연락주세여ㅤㅈㅡㄷjtdbgxvytckjlmftjtfjrfu4,1
...,...,...
54240,어머님이랑생활하고있읍니다 누님도있읍니다형님도있읍니다 결혼을하여따로삽니다저가 2남17...,1
54241,안녕하세요가까운곧에외로운분계실까요 계신다면 만나뵙고 싶습니다부담같은것 없이요...,1
54242,아이둘과.,0
54243,화목 하고 무난하게 자라옴 새로운 가족을 만들어야죠 11111112,0


In [7]:
data = data.loc[~data['conts'].isnull()]

In [8]:
data['label'].value_counts()

0    30116
1    24127
Name: label, dtype: int64

In [9]:
data.shape

(54243, 2)

In [10]:
call_df = data.copy()



In [11]:
call_df = call_df.drop_duplicates()

In [12]:
call_df.isnull().sum()

conts    0
label    0
dtype: int64

In [13]:
call_df.reset_index(drop=True, inplace=True)

In [14]:
label_0 = call_df.loc[call_df['label']==0]
label_1 = call_df.loc[call_df['label']==1]

print(label_0.shape, label_1.shape)

final_call_df = pd.concat([label_0, label_1]).reset_index(drop=True)

(30112, 2) (24072, 2)


In [15]:
# 학습 데이터 : 검증 데이터 8:2

train_data = final_call_df.sample(frac=0.8, random_state=2022)[['conts','label']]
test_data = final_call_df.drop(train_data.index)[['conts','label']]

print(train_data.shape, test_data.shape)

(43347, 2) (10837, 2)


In [16]:
model_name = 'beomi/KcELECTRA-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [17]:
# train dataset 토크나이징
tokenized_train_sentence = tokenizer(
    list(train_data['conts']),
    max_length=128,
    return_tensors='pt',  #pyotorch의 tensor 형태로 return
    padding=True,        #제로패딩 설정
    truncation=True,     # max_length 초과 토큰 truncate
    add_special_tokens=True)  # spcial token 추가


print(tokenized_train_sentence[0])
print(tokenized_train_sentence[0].tokens)
print(tokenized_train_sentence[0].ids)
print(tokenized_train_sentence[0].attention_mask)

Encoding(num_tokens=128, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['[CLS]', '곧', '독립', '##할', '딸', '하나', '있고', ',', '처음', '보는', '이성', '##에', '대한', '불신', '##의', '벽', '##이', '높', '##지만', '현명한', '분이', '##라면', '거짓', '##에', '속지', '않고', '자신', '##에게', '맞는', '짝', '##을', '알아볼', '수', '있을', '거', '##예요', ',', '제', '아무리', '잘난', '사람도', '간절', '##하지', '않으면', '이루어지', '##지', '않', '##기에', '그런', '맘', '##은', '있어야', '겠죠', ',', '남은', '삶', '아끼', '##고', '사랑', '##하며', '우리', '같이', '갑시다', '.', '그래서', '우리는', '동반자', '입', '##니', 's', '##d', '##s', '##80', '##84', '다', '.', '(', '직업', '부동산', '시행', '##사', '##대표', ',', '연봉', '약', '8', '##천', ',', '재산', '20억', '이상', ',', '키', '163', ',', '운동', '##은', '클럽', '##에서', '배드민턴', '##함', ',', '정치적', '##으론', '약간', '보수', ')', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[2, 267, 9217, 4

In [18]:
# test dataset 토크나이징
tokenized_test_sentence = tokenizer(
    list(test_data['conts']),
    max_length=128,
    return_tensors='pt',
    padding=True,
    truncation=True,
    add_special_tokens=True)

In [19]:
print(tokenized_test_sentence[0])
print(tokenized_test_sentence[0].tokens)
print(tokenized_test_sentence[0].ids)
print(tokenized_test_sentence[0].attention_mask)

Encoding(num_tokens=128, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['[CLS]', 'X', '##W', '##U', '##I', '##U', '##N', '##W', '##IC', '##C', '##Q', '##O', '##EE', '##X', '##Q', '##X', '##Z', '##Q', '##Z', '##Q', '##Z', '##I', '##J', '##V', '##T', '##MI', '##EC', '##O', '##E', '##P', '##W', '##X', '##Q', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD

In [20]:
class CustomDataset(torch.utils.data.Dataset) :
    def __init__(self, encodings, labels) :
        self.encodings = encodings
        self.labels = labels
        
    def __len__(self) :
        return len(self.labels)    
        
    def __getitem__(self, idx) :
        item = {key : torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [21]:
# dataset 형성(텐서로 변환)
train_label = train_data['label'].values
test_label = test_data['label'].values

train_dataset = CustomDataset(tokenized_train_sentence, train_label)
test_dataset = CustomDataset(tokenized_test_sentence, test_label)

In [22]:
# # 사전학습 모델 로드
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.to(device)

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.ou

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(50135, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [23]:
# # train option setting
train_arguments = TrainingArguments(
                     output_dir='./',
                     num_train_epochs=5,
                    per_device_train_batch_size=64,
                    per_device_eval_batch_size=64,
                    logging_dir='./logs',
                    logging_steps=500,
                    save_total_limit=2)

In [24]:
# metrics
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, precision_score, recall_score


def compute_metrics(pred) :
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    precision = precision_score(labels, preds) 
    recall = recall_score(labels, preds)
    f1 = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    
    return {
        'accuracy' : acc,
        'f1' : f1,
        'precision' : precision,
        'recall' : recall
    }

In [25]:
%%time

# Train
train = Trainer(
                model=model,
                args=train_arguments,
                train_dataset=train_dataset,
                eval_dataset=test_dataset,
                compute_metrics=compute_metrics)

train.train()

***** Running training *****
  Num examples = 43347
  Num Epochs = 5
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 3390


Step,Training Loss
500,0.065800
1000,0.022200
1500,0.014500
2000,0.009400
2500,0.005500
3000,0.002900


Saving model checkpoint to ./checkpoint-500
Configuration saved in ./checkpoint-500\config.json
Model weights saved in ./checkpoint-500\pytorch_model.bin
Saving model checkpoint to ./checkpoint-1000
Configuration saved in ./checkpoint-1000\config.json
Model weights saved in ./checkpoint-1000\pytorch_model.bin
Saving model checkpoint to ./checkpoint-1500
Configuration saved in ./checkpoint-1500\config.json
Model weights saved in ./checkpoint-1500\pytorch_model.bin
Deleting older checkpoint [checkpoint-500] due to args.save_total_limit
Saving model checkpoint to ./checkpoint-2000
Configuration saved in ./checkpoint-2000\config.json
Model weights saved in ./checkpoint-2000\pytorch_model.bin
Deleting older checkpoint [checkpoint-1000] due to args.save_total_limit
Saving model checkpoint to ./checkpoint-2500
Configuration saved in ./checkpoint-2500\config.json
Model weights saved in ./checkpoint-2500\pytorch_model.bin
Deleting older checkpoint [checkpoint-1500] due to args.save_total_limit


Wall time: 16h 19min 58s


TrainOutput(global_step=3390, training_loss=0.01805992344487733, metrics={'train_runtime': 58797.9235, 'train_samples_per_second': 3.686, 'train_steps_per_second': 0.058, 'total_flos': 1.42563436458624e+16, 'train_loss': 0.01805992344487733, 'epoch': 5.0})

In [26]:
train.evaluate(eval_dataset=test_dataset)

***** Running Evaluation *****
  Num examples = 10837
  Batch size = 64


Trainer is attempting to log a value of "(0.9929299230609274, 0.992517148202037, 0.9927234927234927, None)" of type <class 'tuple'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.039978399872779846,
 'eval_accuracy': 0.9935406477807511,
 'eval_f1': (0.9929299230609274, 0.992517148202037, 0.9927234927234927, None),
 'eval_precision': 0.9929299230609274,
 'eval_recall': 0.992517148202037,
 'eval_runtime': 1026.504,
 'eval_samples_per_second': 10.557,
 'eval_steps_per_second': 0.166,
 'epoch': 5.0}

        {'eval_loss': 0.039978399872779846,
         'eval_accuracy': 0.9935406477807511,
         'eval_f1': (0.9929299230609274, 0.992517148202037, 0.9927234927234927, None),
         'eval_precision': 0.9929299230609274,
         'eval_recall': 0.992517148202037,
         'eval_runtime': 1026.504,
         'eval_samples_per_second': 10.557,
         'eval_steps_per_second': 0.166,
         'epoch': 5.0}

In [28]:
from datetime import datetime

today = str(datetime.today())[:10]
path = f"{today}_call_model.pt"

# save model 
torch.save(model, path)

In [29]:
from datetime import datetime

today = str(datetime.today())[:10]
path = f"{today}_call_model.pt"

# save model 
torch.save(model, path)

In [30]:
# load model
mod = torch.load(f"{today}_call_model.pt")
mod.eval()

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(50135, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [1]:
import re

class Preprocessing():

    def __init__(self, sent):
        self.sent = sent

    def clean_sentence(self):
        try:  # 무성의
            self.sent = re.sub('[^\w\s]', ' ', self.sent).strip()
            self.sent = re.sub('[.,?!ᆢ~]', ', ', self.sent)
            self.sent = re.sub('[ㄱ-ㅎ|ㅏ-ㅣ]', 'ㅋ', self.sent)
            self.sent = re.sub('니다', '니다. ', self.sent)
            self.sent = re.sub('어요', '어요. ', self.sent)
            self.sent = re.sub('\n', ' ', self.sent).strip()

        except:
            print('clean_sentence method fail')
            pass

        return self.sent

    def f_clean_sentence(self):
        try:
            self.sent = re.sub('[^\w\s]', ' ', self.sent).strip()
            self.sent = re.sub('[.,?!ᆢ~]', ', ', self.sent)
            self.sent = self.return_text(self.sent)

        except:
            print('f_clean_sentence method fail')
            pass
        return self.sent

    def sub_num(self, sent):
        hannum_list = ['일', '이', '삼', '사', '오', '육', '륙', '칠', '팔', '구', '십', '영']
        sent = re.sub(r'[0-9]', ' ', sent)

        for i in hannum_list:
            sent = re.sub(rf'{i}', '  ', sent)
        return sent

    def return_target_list(self, pattern, sent):
        word_list = []
        for i in pattern.finditer(sent):
            target_word = sent[i.start(): i.end()]
            word_list.append(target_word)
        return word_list

    def return_text(self, sent):

        nam = '[남]+'
        nyeo = '[녀]+'
        yeo = '[여]+'
        son = '[아들]+'
        ddal = '[딸]+'
        num = '[\s]*(\d){1,2}[\s]*'
        han_num = '[일이삼사오육륙칠팔구십]+'

        person_list = [nam, son, nyeo, yeo, ddal]
        num_list = [num, han_num]

        for person_pattern in person_list:
            for num_pattern in num_list:
                pattern_list = [person_pattern + num_pattern, num_pattern + person_pattern]
                target_list = []

                for pattern in pattern_list:
                    add_pattern = re.compile(pattern)
                    m = add_pattern.findall(sent)
                    if m != []:
                        target_words = self.return_target_list(add_pattern, sent)
                        target_list += target_words
                    else:
                        pass

                for i in target_list:
                    sent = sent.replace(i, self.sub_num(i))
        return sent

In [2]:
import torch.nn.functional as F

class predictModel():

    def __init__(self, model_path):
        # load model
        self.model_path = model_path
        self.model = torch.load(model_path)
        # set device
        self.device = torch.device('cpu')
        # load tokenizer
        model_name = 'beomi/KcELECTRA-base'
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def predict_sentence(self, sent):
        self.model.eval()
        Pr = Preprocessing(sent)

        if 'call' in str(self.model_path):
            sent = Pr.f_clean_sentence()

        else:
            sent = Pr.clean_sentence()

        # tokenizing
        tokenized_sent = self.tokenizer(
            sent,
            return_tensors='pt',
            truncation=True,
            add_special_tokens=True,
            max_length=512
        )
        tokenized_sent.to(self.device)

        # prediction
        with torch.no_grad():
            outputs = self.model(
                input_ids=tokenized_sent['input_ids'],
                attention_mask=tokenized_sent['attention_mask'],
                token_type_ids=tokenized_sent['token_type_ids']
            )

        # result
        per = int(str(np.array(F.softmax(outputs[0][0], dim=0).detach().cpu())[1] * 100).split('.')[0])

        if len(sent) <= 5:
            # 10자 이하는 연락처 로 탐지하지 않음
            return 1, int(per)

        result = outputs[0].detach().cpu().argmax(-1)

        if int(result) == 0:
            return int(result), int(per)

        return int(result), int(per)

In [9]:
from datetime import datetime

today = str(datetime.today())[:10]

In [10]:
call_model = predictModel(model_path = f"{today}_call_model.pt")